<a href="https://colab.research.google.com/github/Hoangnhat4869/TrafficLightDetection/blob/main/TrafficLightDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1. Install YOLOv5 and some requirements

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16567, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 16567 (delta 22), reused 29 (delta 13), pack-reused 16522
Receiving objects: 100% (16567/16567), 15.08 MiB | 19.07 MiB/s, done.
Resolving deltas: 100% (11379/11379), done.


In [ ]:
# import shutil
# shutil.rmtree('../yolov5', ignore_errors=True)
# import os
# print(os.getcwd())

In [ ]:
cd yolov5

/content/yolov5


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

### 2. Train model

In [ ]:
!pip install pybboxes
import xml.etree.ElementTree as ET
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import pandas as pd
import pybboxes as pbx
from collections import defaultdict
import shutil

In [ ]:
# make labels dir in data folder of YOLOv5
output_path = '/content/yolov5'
labels_dir = f'{output_path}/data/labels'
if os.path.exists(labels_dir):
  shutil.rmtree(labels_dir)
os.mkdir(labels_dir)

In [ ]:
import os
import shutil
from random import shuffle

# input_path = '/content/drive/MyDrive/TrafficLightDetection/dataset'
# images_path = input_path + '/images'
# labels_path = labels_dir  # Directory having labels in .txt for mat

In [ ]:
# create train, val folders in data directory of YOLOv5
train_dir = output_path + '/data/train'
val_dir = output_path + '/data/valid'

if os.path.exists(train_dir): shutil.rmtree(train_dir)
if os.path.exists(val_dir): shutil.rmtree(val_dir)

os.mkdir(train_dir)
os.mkdir(val_dir)

# train, val each containing images and labels folders
os.mkdir(train_dir + '/images')
os.mkdir(train_dir + '/labels')
os.mkdir(val_dir + '/images')
os.mkdir(val_dir + '/labels')

In [ ]:
# def copy_files(images_path, labels_path, destination_path, imgfiles, lblfiles):
#   for filename in imgfiles:
#     src = images_path + '/' + filename
#     dst = destination_path + '/images'
#     shutil.copy(src, dst)
#     # print(src)

#   for filename in lblfiles:
#     src = labels_path + '/' + filename
#     dst = destination_path + '/labels'
#     shutil.copy(src, dst)
#     # print('2.', src)

input_path = '/content/drive/MyDrive/TrafficLightDetection/dataset'

images_train_path = input_path + '/train/images'
labels_train_path = input_path + '/train/labels'
# images_train_files = os.listdir(images_train_path)
# labels_train_files = os.listdir(labels_train_path)

images_val_path = input_path + '/valid/images'
labels_val_path = input_path + '/valid/labels'
# images_val_files = os.listdir(images_val_path)
# labels_val_files = os.listdir(labels_val_path)
for file in os.listdir(images_train_path):
  shutil.copy(images_train_path + '/' + file, train_dir + '/images')
for file in os.listdir(labels_train_path):
  shutil.copy(labels_train_path + '/' + file, train_dir + '/labels')
for file in os.listdir(images_val_path):
  shutil.copy(images_val_path + '/' + file, val_dir + '/images')
for file in os.listdir(labels_val_path):
  shutil.copy(labels_val_path + '/' + file, val_dir + '/labels')
# copy_files(images_train_path, labels_train_path, train_dir, images_train_files, labels_train_files)
# copy_files(images_val_path, labels_val_path, val_dir, images_val_files, labels_val_files)

In [ ]:
# with open(f'{output_path}/data/sign_data.yaml', 'w') as f:
#   f.write('train: ../data/train/images\n')
#   f.write('val: ../data/val/images\n')
#   f.write('nc: 15\n')
#   f.write(f'names: {names}')
yamlfile = input_path + '/data.yaml'
yamloutput = output_path + '/data'
shutil.copy(yamlfile, yamloutput)

In [ ]:
epochs = 20
!python train.py --img 640 --batch 16 --epochs {epochs} --data data.yaml --weights yolov5s.pt

2024-04-23 06:36:28.457729: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 06:36:28.457781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 06:36:28.459190: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optim

In [ ]:
import shutil
import os
shutil.copy('/content/yolov5/runs/train/exp8/weights/best.pt', '/content/drive/MyDrive/TrafficLightDetection')

'/content/drive/MyDrive/TrafficLightDetection/best.pt'

In [ ]:
## Results stored in runs/train
## get the last stored result
exp = sorted(os.listdir(output_path + '/runs/train'))[-1]
exp_path = output_path + '/runs/train/' + exp
os.listdir(exp_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/yolov5/runs/train'

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

img_path = exp_path + '/val_batch0_pred.jpg'
img = Image.open(img_path)
plt.figure(figsize=(15,15))
plt.imshow(img)

In [13]:
!python detect.py --weights /content/drive/MyDrive/TrafficLightDetection/best.pt --img 416 --conf 0.4 --source /content/drive/MyDrive/TrafficLightDetection/test_images/test.jpg

detect: weights=['/content/drive/MyDrive/TrafficLightDetection/best.pt'], source=/content/drive/MyDrive/TrafficLightDetection/test_images/test.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-304-g22361691 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7050580 parameters, 0 gradients, 15.9 GFLOPs
image 1/1 /content/drive/MyDrive/TrafficLightDetection/test_images/test.jpg: 288x416 2 Red Lights, 217.7ms
Speed: 1.5ms pre-process, 217.7ms inference, 31.9ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp
